## Import Library and Config

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import polars as pl
import os
from pathlib import Path

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import StratifiedGroupKFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import VotingClassifier

from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler

import lightgbm as lgb

import time
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif, VarianceThreshold

## Data Loading and Setting

In [2]:
# Data Load
df_train = pd.read_csv('/kaggle/input/isic-2024-challenge/train-metadata.csv')
df_test  = pd.read_csv('/kaggle/input/isic-2024-challenge/test-metadata.csv')

# Pandas 출력 Option 설정 
pd.set_option('display.max_columns',100 , ) # 최대 100개의 열을 표시되도록 설정
pd.set_option('display.width', 200) # 최대 200자까지 한줄로 표시되도록 설정

## Pre-trained Models, Generating Image Prediction Results

Pre-Train EffNet, Vision Transformer Model를 사용해 Image 예측 결과를 생성한다. 

In [3]:
!python /kaggle/input/isic-script-inference-effnetv1b0-f313ae/main.py /kaggle/input/isic-pytorch-training-baseline-image-only/AUROC0.5171_Loss0.3476_epoch35.bin
!mv submission.csv submission_effnetv1b0.csv

/opt/conda/lib/python3.10/site-packages/albumentations/check_version.py:49: UserWarning: Error fetching version info <urlopen error [Errno -3] Temporary failure in name resolution>
  data = fetch_version_info()
BEST_WEIGHT = /kaggle/input/isic-pytorch-training-baseline-image-only/AUROC0.5171_Loss0.3476_epoch35.bin
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.00it/s]


In [4]:
!python /kaggle/input/isic-script-inference-eva02/main.py /kaggle/input/isic-pytorch-training-baseline-eva02/AUROC0.5177_Loss0.2829_epoch7.bin
!mv submission.csv submission_eva02.csv

/opt/conda/lib/python3.10/site-packages/albumentations/check_version.py:49: UserWarning: Error fetching version info <urlopen error [Errno -3] Temporary failure in name resolution>
  data = fetch_version_info()
BEST_WEIGHT = /kaggle/input/isic-pytorch-training-baseline-eva02/AUROC0.5177_Loss0.2829_epoch7.bin
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  2.20it/s]


## Path and Parameter Setting

In [5]:
root = Path('/kaggle/input/isic-2024-challenge')

train_path = root / 'train-metadata.csv'
test_path = root / 'test-metadata.csv'
subm_path = root / 'sample_submission.csv'

id_col = 'isic_id'
target_col = 'target'
group_col = 'patient_id'

err = 1e-5 # 계산 시 0으로 나누는 것을 방지하기 위해 사용하는 작은 상수
sampling_ratio = 0.01 # Class 불균형을 조정하기 위한 Sampling 비율
seed = 42 # 난수 생성의 일관성을 유지하기 위한 Random Seed 값 

## Feature Column Setting

In [6]:
# 기존의 수치형 Features의 List
num_cols = [
    'age_approx',                        
    'clin_size_long_diam_mm',            
    'tbp_lv_A',                          
    'tbp_lv_Aext',                       
    'tbp_lv_B',                          
    'tbp_lv_Bext',                       
    'tbp_lv_C',                          
    'tbp_lv_Cext',                       
    'tbp_lv_H',                          
    'tbp_lv_Hext',                       
    'tbp_lv_L',                          
    'tbp_lv_Lext',                       
    'tbp_lv_areaMM2',                    
    'tbp_lv_area_perim_ratio',           
    'tbp_lv_color_std_mean',             
    'tbp_lv_deltaA',                     
    'tbp_lv_deltaB',                     
    'tbp_lv_deltaL',                     
    'tbp_lv_deltaLB',                    
    'tbp_lv_deltaLBnorm',                
    'tbp_lv_eccentricity',               
    'tbp_lv_minorAxisMM',                
    'tbp_lv_nevi_confidence',            
    'tbp_lv_norm_border',                
    'tbp_lv_norm_color',                 
    'tbp_lv_perimeterMM',                
    'tbp_lv_radial_color_std_max',       
    'tbp_lv_stdL',                       
    'tbp_lv_stdLExt',                    
    'tbp_lv_symm_2axis',                 
    'tbp_lv_symm_2axis_angle',           
    'tbp_lv_x',                          
    'tbp_lv_y',                          
    'tbp_lv_z',                          
]

In [7]:
# 기존 Features를 조합해서 만든 새로운 Numerical Features
new_num_cols = [
    'lesion_size_ratio',             # tbp_lv_minorAxisMM      / clin_size_long_diam_mm
    'lesion_shape_index',            # tbp_lv_areaMM2          / tbp_lv_perimeterMM ** 2
    'hue_contrast',                  # abs(tbp_lv_H            - tbp_lv_Hext)
    'luminance_contrast',            # abs(tbp_lv_L            - tbp_lv_Lext)
    'lesion_color_difference',       # sqrt(tbp_lv_deltaA **2  + tbp_lv_deltaB ** 2 + tbp_lv_deltaL ** 2)  
    'border_complexity',             # tbp_lv_norm_border      + tbp_lv_symm_2axis
    'color_uniformity',              # tbp_lv_color_std_mean   / tbp_lv_radial_color_std_max

    'position_distance_3d',          # sqrt(tbp_lv_x ** 2 + tbp_lv_y ** 2 + tbp_lv_z ** 2)
    'perimeter_to_area_ratio',       # tbp_lv_perimeterMM      / tbp_lv_areaMM2
    'area_to_perimeter_ratio',       # tbp_lv_areaMM2          / tbp_lv_perimeterMM
    'lesion_visibility_score',       # tbp_lv_deltaLBnorm      + tbp_lv_norm_color
    'symmetry_border_consistency',   # tbp_lv_symm_2axis       * tbp_lv_norm_border
    'consistency_symmetry_border',   # tbp_lv_symm_2axis       * tbp_lv_norm_border / (tbp_lv_symm_2axis + tbp_lv_norm_border)

    'color_consistency',             # tbp_lv_stdL             / tbp_lv_Lext
    'consistency_color',             # tbp_lv_stdL * tbp_lv_Lext / (tbp_lv_stdL + tbp_lv_Lext)
    'size_age_interaction',          # clin_size_long_diam_mm  * age_approx
    'hue_color_std_interaction',     # tbp_lv_H                * tbp_lv_color_std_mean
    'lesion_severity_index',         # (tbp_lv_norm_border     + tbp_lv_norm_color + tbp_lv_eccentricity) / 3
    'shape_complexity_index',        # border_complexity       + lesion_shape_index
    'color_contrast_index',          # tbp_lv_deltaA + tbp_lv_deltaB + tbp_lv_deltaL + tbp_lv_deltaLBnorm

    'log_lesion_area',               # log(tbp_lv_areaMM2 + 1)
    'normalized_lesion_size',        # clin_size_long_diam_mm  / age_approx
    'mean_hue_difference',           # (tbp_lv_H + tbp_lv_Hext) / 2
    'std_dev_contrast',              # sqrt((tbp_lv_deltaA ** 2 + tbp_lv_deltaB ** 2 + tbp_lv_deltaL ** 2) / 3)
    'color_shape_composite_index',   # (tbp_lv_color_std_mean + tbp_lv_area_perim_ratio + tbp_lv_symm_2axis) / 3
    'lesion_orientation_3d',         # arctan2(tbp_lv_y, tbp_lv_x)
    'overall_color_difference',      # (tbp_lv_deltaA + tbp_lv_deltaB + tbp_lv_deltaL) / 3
]

## Categorical and Special Column Settings

'cat_cols' : Categorical Data Columns로 OneHot-Encoding이 필요하다. <br>
'norm_cols' : 환자 기준으로 정규화 된 Feature들의 Columns로 각 Numerical Feature는 환자 Group 내에서 정규화된다. <br>
'special_cols' : 특수한 Feature들의 List <br>
'image_cols' : Image Model의 예측 결과로 생성된 Columns

In [8]:
# Categorical Columns (Features)
cat_cols = ['sex', 'anatom_site_general', 'tbp_tile_type', 'tbp_lv_location', 'tbp_lv_location_simple', 'attribution']

# Normalized Columns, 환자 기준으로 정규화 된 Numerical Features
norm_cols = [f'{col}_patient_norm' for col in num_cols + new_num_cols]

# Special Columns, 특수한 Features 
special_cols = ['count_per_patient']

# Image Columns, Image Model에서 생성된 예측 결과를 추가한 Features
image_col = ['target_effnetv1b0', 'target_eva02']

In [9]:
# All feature columns
feature_cols = num_cols + new_num_cols + cat_cols + norm_cols + special_cols + image_col 

## Data Preprocess Function

Categorical Variables를 OneHot-Encoding해 'new_cat_cols' Column에 저장한다. <br>
Image Model의 예측 결과를 'target_effnetv1b0', 'target_eva02'라는 새로운 Column로 Train Data, Test Data에 추가한다. <br>
기존 'cat_cols'을 제거하고 Encoding 된 새로운 Columns 'new_cat_cols'로 대체해 'feature_cols'에 Update 한다. 

In [10]:
def preprocess(df_train, df_test):
    global cat_cols, feature_cols 
    
    # OneHot-Encoding for Categorical Features
    encoder = OneHotEncoder(sparse_output = False, dtype = np.int32, handle_unknown = 'ignore')
    encoder.fit(df_train[cat_cols])
    
    new_cat_cols = [f'onehot_{i}' for i in range(len(encoder.get_feature_names_out()))]

    df_train[new_cat_cols] = encoder.transform(df_train[cat_cols])
    df_train[new_cat_cols] = df_train[new_cat_cols].astype('category')

    df_test[new_cat_cols] = encoder.transform(df_test[cat_cols])
    df_test[new_cat_cols] = df_test[new_cat_cols].astype('category')
    
    # Add Image Prediction Features
    # effnetv1b0
    df_eff = pd.read_csv("/kaggle/input/isic-inference-effnetv1b0-for-training-data/train_effnetv1b0.csv")
    df_train["target_effnetv1b0"] = df_eff["target_effnetv1b0"]

    df_eff = pd.read_csv("/kaggle/input/isic-tabular-model-image-model-features/submission_effnetv1b0.csv")
    df_test["target_effnetv1b0"] = df_eff["target"]
    
    # eva02
    df_eva = pd.read_csv("/kaggle/input/isic-inference-eva02-for-training-data/train_eva02.csv")
    df_train["target_eva02"] = df_eva["target_eva02"]

    df_eva = pd.read_csv("/kaggle/input/isic-tabular-model-image-model-features/submission_eva02.csv")
    df_test["target_eva02"] = df_eva["target"]

    # Remove Original Categorical Columns and Replace with Encoded Ones
    for col in cat_cols:
        feature_cols.remove(col)

    feature_cols.extend(new_cat_cols)
    cat_cols = new_cat_cols
    
    # Ensure image columns are in feature_cols
    for col in image_col:
        if col not in feature_cols:
            feature_cols.append(col)
    
    return df_train, df_test

## Data Read Function

'pl.read_csv'로 Data를 읽어온다. <br>
새로운 Numerical Feature, 특수 Feature, 환자당 Sample 수와 정규화 된 Feature를 생성한다. <br>
결측치는 중앙값으로 대체한다. <br>
Categorical Data를 'Catecorical'로 변환해 Memory를 절약한다. <br>
DataFrame을 Pandas 형식으로 변환 후 반환한다. 

In [11]:
 def read_data(path):
    return (
        pl.read_csv(path)
        .with_columns(
            pl.col('age_approx').cast(pl.String).replace('NA', np.nan).cast(pl.Float64),
        )
        .with_columns(
            pl.col(pl.Float64).fill_nan(pl.col(pl.Float64).median()), 
        )
        .with_columns(
            lesion_size_ratio              = pl.col('tbp_lv_minorAxisMM') / pl.col('clin_size_long_diam_mm'),
            lesion_shape_index             = pl.col('tbp_lv_areaMM2') / (pl.col('tbp_lv_perimeterMM') ** 2),
            hue_contrast                   = (pl.col('tbp_lv_H') - pl.col('tbp_lv_Hext')).abs(),
            luminance_contrast             = (pl.col('tbp_lv_L') - pl.col('tbp_lv_Lext')).abs(),
            lesion_color_difference        = (pl.col('tbp_lv_deltaA') ** 2 + pl.col('tbp_lv_deltaB') ** 2 + pl.col('tbp_lv_deltaL') ** 2).sqrt(),
            border_complexity              = pl.col('tbp_lv_norm_border') + pl.col('tbp_lv_symm_2axis'),
            color_uniformity               = pl.col('tbp_lv_color_std_mean') / (pl.col('tbp_lv_radial_color_std_max') + err),
        )
        .with_columns(
            position_distance_3d           = (pl.col('tbp_lv_x') ** 2 + pl.col('tbp_lv_y') ** 2 + pl.col('tbp_lv_z') ** 2).sqrt(),
            perimeter_to_area_ratio        = pl.col('tbp_lv_perimeterMM') / pl.col('tbp_lv_areaMM2'),
            area_to_perimeter_ratio        = pl.col('tbp_lv_areaMM2') / pl.col('tbp_lv_perimeterMM'),
            lesion_visibility_score        = pl.col('tbp_lv_deltaLBnorm') + pl.col('tbp_lv_norm_color'),
            combined_anatomical_site       = pl.col('anatom_site_general') + '_' + pl.col('tbp_lv_location'),
            symmetry_border_consistency    = pl.col('tbp_lv_symm_2axis') * pl.col('tbp_lv_norm_border'),
            consistency_symmetry_border    = pl.col('tbp_lv_symm_2axis') * pl.col('tbp_lv_norm_border') / (pl.col('tbp_lv_symm_2axis') + pl.col('tbp_lv_norm_border')),
        )
        .with_columns(
            color_consistency              = pl.col('tbp_lv_stdL') / pl.col('tbp_lv_Lext'),
            consistency_color              = pl.col('tbp_lv_stdL') * pl.col('tbp_lv_Lext') / (pl.col('tbp_lv_stdL') + pl.col('tbp_lv_Lext')),
            size_age_interaction           = pl.col('clin_size_long_diam_mm') * pl.col('age_approx'),
            hue_color_std_interaction      = pl.col('tbp_lv_H') * pl.col('tbp_lv_color_std_mean'),
            lesion_severity_index          = (pl.col('tbp_lv_norm_border') + pl.col('tbp_lv_norm_color') + pl.col('tbp_lv_eccentricity')) / 3,
            shape_complexity_index         = pl.col('border_complexity') + pl.col('lesion_shape_index'),
            color_contrast_index           = pl.col('tbp_lv_deltaA') + pl.col('tbp_lv_deltaB') + pl.col('tbp_lv_deltaL') + pl.col('tbp_lv_deltaLBnorm'),
        )
        .with_columns(
            log_lesion_area                = (pl.col('tbp_lv_areaMM2') + 1).log(),
            normalized_lesion_size         = pl.col('clin_size_long_diam_mm') / pl.col('age_approx'),
            mean_hue_difference            = (pl.col('tbp_lv_H') + pl.col('tbp_lv_Hext')) / 2,
            std_dev_contrast               = ((pl.col('tbp_lv_deltaA') ** 2 + pl.col('tbp_lv_deltaB') ** 2 + pl.col('tbp_lv_deltaL') ** 2) / 3).sqrt(),
            color_shape_composite_index    = (pl.col('tbp_lv_color_std_mean') + pl.col('tbp_lv_area_perim_ratio') + pl.col('tbp_lv_symm_2axis')) / 3,
            lesion_orientation_3d          = pl.arctan2(pl.col('tbp_lv_y'), pl.col('tbp_lv_x')),
            overall_color_difference       = (pl.col('tbp_lv_deltaA') + pl.col('tbp_lv_deltaB') + pl.col('tbp_lv_deltaL')) / 3,
        )
        .with_columns(
            symmetry_perimeter_interaction = pl.col('tbp_lv_symm_2axis') * pl.col('tbp_lv_perimeterMM'),
            comprehensive_lesion_index     = (pl.col('tbp_lv_area_perim_ratio') + pl.col('tbp_lv_eccentricity') + pl.col('tbp_lv_norm_color') + pl.col('tbp_lv_symm_2axis')) / 4,
            color_variance_ratio           = pl.col('tbp_lv_color_std_mean') / pl.col('tbp_lv_stdLExt'),
            border_color_interaction       = pl.col('tbp_lv_norm_border') * pl.col('tbp_lv_norm_color'),
            border_color_interaction_2     = pl.col('tbp_lv_norm_border') * pl.col('tbp_lv_norm_color') / (pl.col('tbp_lv_norm_border') + pl.col('tbp_lv_norm_color')),
            size_color_contrast_ratio      = pl.col('clin_size_long_diam_mm') / pl.col('tbp_lv_deltaLBnorm'),
            age_normalized_nevi_confidence = pl.col('tbp_lv_nevi_confidence') / pl.col('age_approx'),
            age_normalized_nevi_confidence_2 = (pl.col('clin_size_long_diam_mm')**2 + pl.col('age_approx')**2).sqrt(),
            color_asymmetry_index          = pl.col('tbp_lv_radial_color_std_max') * pl.col('tbp_lv_symm_2axis'),
        )
        .with_columns(
            volume_approximation_3d        = pl.col('tbp_lv_areaMM2') * (pl.col('tbp_lv_x')**2 + pl.col('tbp_lv_y')**2 + pl.col('tbp_lv_z')**2).sqrt(),
            color_range                    = (pl.col('tbp_lv_L') - pl.col('tbp_lv_Lext')).abs() + (pl.col('tbp_lv_A') - pl.col('tbp_lv_Aext')).abs() + (pl.col('tbp_lv_B') - pl.col('tbp_lv_Bext')).abs(),
            shape_color_consistency        = pl.col('tbp_lv_eccentricity') * pl.col('tbp_lv_color_std_mean'),
            border_length_ratio            = pl.col('tbp_lv_perimeterMM') / (2 * np.pi * (pl.col('tbp_lv_areaMM2') / np.pi).sqrt()),
            age_size_symmetry_index        = pl.col('age_approx') * pl.col('clin_size_long_diam_mm') * pl.col('tbp_lv_symm_2axis'),
            index_age_size_symmetry        = pl.col('age_approx') * pl.col('tbp_lv_areaMM2') * pl.col('tbp_lv_symm_2axis'),
        )
        .with_columns(
            ((pl.col(col) - pl.col(col).mean().over('patient_id')) / (pl.col(col).std().over('patient_id') + err)).alias(f'{col}_patient_norm') for col in (num_cols + new_num_cols)
        )
        .with_columns(
            count_per_patient = pl.col('isic_id').count().over('patient_id'),
        )
        .with_columns(
            pl.col(cat_cols).cast(pl.Categorical),
        )
        .to_pandas()
        .set_index(id_col)
    )

## Data Read & Pre-Processing

In [12]:
# Data Read
df_train = read_data(train_path)
df_test = read_data(test_path)
df_subm = pd.read_csv(subm_path, index_col = id_col)

# Data Pre-Processing
df_train, df_test = preprocess(df_train, df_test)

## LightGBM Model Hyperarameter Setting

In [13]:
lgb_params = {
    'objective':        'binary',
    'verbosity':        -1,
    'n_iter':           200,
    'boosting_type':    'gbdt',
    'random_state':     seed,
    'lambda_l1':        0.08758718919397321, 
    'lambda_l2':        0.0039689175176025465, 
    'learning_rate':    0.03231007103195577, 
    'max_depth':        4, 
    'num_leaves':       103, 
    'colsample_bytree': 0.8329551585827726, 
    'colsample_bynode': 0.4025961355653304, 
    'bagging_fraction': 0.7738954452473223, 
    'bagging_freq':     4, 
    'min_data_in_leaf': 85, 
    'scale_pos_weight': 2.7984184778875543,
}

## LightGBM Model Pipline

Pipline을 사용해 Data Sampling과 Model Train을 연결한다. <br>
Oversampling, Undersampling을 통해 Class 불균형을 조정한 후 LightGBM Model을 Train한다. 

In [14]:
lgb_model = Pipeline([
    ('sampler_1', RandomOverSampler(sampling_strategy = 0.003, random_state = seed)),
    ('sampler_2', RandomUnderSampler(sampling_strategy = sampling_ratio, random_state = seed)),
    ('classifier', lgb.LGBMClassifier(**lgb_params)),
])

## StratifiedGroupKFold

StratifiedGroupKFold(교차 검증)을 사용해 5-Fold 교차 검증을 설정한다. <br>
이후 'cross_val_score' Function을 사용해 Model의 성능을 평가하고, ROC AUC Score의 평균을 출력한다.

In [15]:
# 'X', 'y' = Model에 사용할 Feature와 Target Data를 설정한다
X = df_train[feature_cols]
y = df_train[target_col]
# 'group' = 환자 ID 별로 Group화 된 Data를 설정한다. 
groups = df_train[group_col]
cv = StratifiedGroupKFold(5, shuffle = True, random_state = seed)

val_score = cross_val_score(
    estimator = lgb_model, 
    X = X, y = y, 
    cv = cv, 
    groups = groups,
    scoring = 'roc_auc',  # ROC AUC Score
)

print(f'Cross-validation AUC score: {np.mean(val_score):.4f}')

Cross-validation AUC score: 0.9643


In [16]:
lgb_model.fit(X, y)

Pipeline(steps=[('sampler_1',
                 RandomOverSampler(random_state=42, sampling_strategy=0.003)),
                ('sampler_2',
                 RandomUnderSampler(random_state=42, sampling_strategy=0.01)),
                ('classifier',
                 LGBMClassifier(bagging_fraction=0.7738954452473223,
                                bagging_freq=4,
                                colsample_bynode=0.4025961355653304,
                                colsample_bytree=0.8329551585827726,
                                lambda_l1=0.08758718919397321,
                                lambda_l2=0.0039689175176025465,
                                learning_rate=0.03231007103195577, max_depth=4,
                                min_data_in_leaf=85, n_iter=200, num_leaves=103,
                                objective='binary', random_state=42,
                                scale_pos_weight=2.7984184778875543,
                                verbosity=-1))])

**Data Sampling 전략 (RandomOverSampler, RandomUnderSampler)** <br>
Data의 Class 불균형 문제를 해결하기 위해 먼저 소수 Class를 Oversampling하고, 이후 다수 Class를 Undersmapling하여 Data 균형을 맞추고 있다. <br>
**LightGBM Model (LGBMClassifier)** <br>
다양한 하이퍼파라미터 설정을 통해 Learning Rate, Regularization, Tree의 Depth, Node 수 등을 조절하여 Model의 성능을 최적화하고 있다. <br> 
scale_pos_weight 및 Sampling 전략이 Class 불균형 문제를 보정하도록 설계되어 있다.
    

### Pipeline

Pipeline은 여러 Data 처리 단계와 Model을 순차적으로 결합하여 관리할 수 있는 도구이다. <br>
이 Pipeline은 Data의 Class 불균형을 먼저 조정한 후에 LightGBM 분류기를 사용하여 Train을 진행한다. <br>
단계별로 Oversampling과 Undersmapling을 조정하여 Class 비율을 변경하고, LGBMClassifier는 이러한 전처리된 Data를 사용하여 Train한다.

- **sampler_1 : RandomOverSampler**
    - Class 불균형 문제를 해결하기 위해 Sampling 전략으로 Oversampling을 수행하는 단계이다.
- **sampler_2 : RandomUnderSampler**
    - Oversampling 이후에 다시 Undersmapling을 수행하는 단계이다.
- **classifier: LGBMClassifier**
    - LightGBM Model을 사용하여 분류를 수행하는 단계이다.

### RandomOverSampler

RandomOverSampler는 Oversampling을 통해 소수 Class의 Data 수를 증가시키는 도구이다. <br>
이 단계는 소수 Class의 Data 수가 적기 때문에 Data의 불균형을 완화하기 위해 사용된다. <br> 
Oversampling을 통해 소수 Class의 Data 수를 늘리고, Train 시 해당 Class가 과소평가되지 않도록 한다.

- **random_state = 42**    
    Random Seed를 설정하여 결과의 재현성을 보장한다.    
- **sampling_strategy = 0.003**    
    소수 Class의 샘플 수를 증가시키는 비율을 나타낸다.    
    소수 Class가 전체 Data의 약 0.3%가 될 때까지 Sampling을 수행한다는 의미이다.
    
### RandomUnderSampler

RandomUnderSampler는 Oversampling된 Data를 다시 Undersmapling하여 다수 Class의 Data 수를 줄이는 도구이다. <br>
Oversampling 후 Data의 양이 매우 많아질 수 있으므로, Undersmapling을 통해 다시 다수 Class의 Data를 줄여서 Data 양을 조정하고 Train 효율을 높인다.

- **random_state = 42**  
    Random Seed를 설정하여 결과의 재현성을 보장한다.    
- **sampling_strategy = 0.01**    
    다수 Class의 샘플 수를 감소시키는 비율을 나타낸다.    
    다수 Class가 전체 Data의 약 1%가 될 때까지 Sampling을 수행한다는 의미이다.
    

### LGBMClassifier

LGBMClassifier는 LightGBM(Light Gradient Boosting Machine) 분류 Model이다. <br>
주어진 Hyperparameter를 통해 Train이 수행된다. <br>
이 Model은 여러 하이퍼파라미터를 통해 Class 불균형 문제를 해결하고, Learning Rate와 Regularization, Tree의 Depth, Node 수 등을 조절하여 Model의 성능을 최적화하려고 한다. <br>
Class 불균형을 해결하기 위해 scale_pos_weight가 설정되어 있고, 여러 Regularization 기법을 통해 Model이 과적합되지 않도록 하고 있다.

- **bagging_fraction = 0.7739**    
    Sampling 비율을 설정하여 일부 Data만 사용하여 Model을 Train시킨다. 이는 과적합을 방지하기 위한 방법이다.    
- **bagging_freq = 4**    
    몇 번째 반복마다 Sampling을 수행할지를 설정한다. 4이면 매 4번째 반복마다 Sampling이 수행된다.    
- **colsample_bynode = 0.4026, colsample_bytree = 0.8329**    
    각 Tree 및 각 Node에서 사용할 Feature의 비율을 설정한다. 이는 Model의 다변성을 높이고 과적합을 방지하기 위한 설정이다.    
- **lambda_l1=0.0876, lambda_l2 = 0.0039**   
    L1 및 L2 Regularization 항의 강도를 설정하여 Model의 복잡성을 제어한다.   
- **learning_rate = 0.0323**    
    Train 속도를 결정하는 Learning Rate이다. 값이 작을수록 Train이 천천히 진행되며, 이는 성능 향상과 일반화에 도움이 될 수 있다.
- **max_depth = 4**    
    Tree의 최대 Depth를 설정한다. 이 값이 작을수록 Model의 복잡성이 줄어든다.    
- **min_data_in_leaf = 85**    
    각 Leaf Node에 포함될 최소 Data 포인트 수를 설정한다. 이는 Model의 과적합을 방지하기 위한 방법이다.    
- **n_iter = 200**    
    Train 반복 수를 설정한다.    
- **num_leaves = 103**    
    하나의 Tree에서 사용할 최대 Leaf 수이다. 이 값이 커지면 Tree의 복잡도가 증가할 수 있다.    
- **objective = 'binary'**    
    Binary Classification 문제를 해결하는 것을 목표로 한다.    
- **random_state = 42**    
    Random Seed를 설정하여 결과의 재현성을 보장한다.    
- **scale_pos_weight = 2.7984**    
    Class 불균형을 보정하기 위한 가중치이다. 이는 소수 Class의 가중치를 늘려주는 역할을 한다.    
- **verbosity = -1**   
    Log 출력을 제어한다. 1은 출력하지 않음을 의미한다

## Submission

In [17]:
df_subm['target'] = lgb_model.predict_proba(df_test[feature_cols])[:, 1]
df_subm.to_csv('submission.csv')
df_subm.head()

,target
isic_id,
ISIC_0015657,0.788470
ISIC_0015729,0.413181
ISIC_0015740,0.652616


## Feature Inportance

LightGBM Model의 Feature 중요도는 각 Feature가 Model의 예측에 얼마나 기여했는지를 나타낸다. 

In [18]:
'''
pd.set_option('display.max_rows', None)  # Row
pd.set_option('display.max_columns', None)  # Column
pd.set_option('display.width', 1000)  # Output Width
'''

lgb_model.named_steps['classifier'].fit(X, y)  # Model Re-Train
importance = lgb_model.named_steps['classifier'].feature_importances_
feature_importance = pd.DataFrame({'feature': X.columns, 'importance': importance}).sort_values(by='importance', ascending=False)

print(feature_importance)

                               feature  importance
122                  count_per_patient          81
61             age_approx_patient_norm          70
69               tbp_lv_H_patient_norm          62
36                        hue_contrast          49
102  position_distance_3d_patient_norm          46
..                                 ...         ...
156                          onehot_31           0
159                          onehot_34           0
161                          onehot_36           0
162                          onehot_37           0
163                          onehot_38           0

[172 rows x 2 columns]


## Feature Vector and Predicted Prob

각 ID마다 Feature Vector와 예측 확률을 계산한다. <br>
'predicted_prob' = 예측된 확률 <br>
'feature_vector' = Feature Vector

In [19]:
# Predict and Output Feature Vectors
# 각 ID마다 추출된 Feature Vector는 X.value로 Vectorizing 되어 'featrue_vector' Column에 저장된다.
# lgb_model.predict_proba(X)[:, 1]로 예측된 확률을 추출해 'predicted_prob' Column에 저장한다. 
df_train['predicted_prob'] = lgb_model.predict_proba(X)[:, 1]
# list(X.values)를 사용해 각 Feature Vector를 List 형식으로 저장한다. 
df_train['feature_vector'] = list(X.values) 

In [20]:
# 'target' : 실제 Target 값
# 'predicted_prob' : 예측된 확률
# 'feature_vector' : ID마다 Feature Vector(List)
feature_output = df_train[['target', 'predicted_prob', 'feature_vector']]

In [21]:
feature_output

,target,predicted_prob,feature_vector
isic_id,,,
ISIC_0015670,0,0.000155,"[60.0, 3.04, 20.2444222995213, 16.261975179121..."
ISIC_0015845,0,0.447526,"[60.0, 1.1, 31.71257, 25.36474, 26.331, 24.549..."
ISIC_0015864,0,0.000163,"[60.0, 3.4, 22.57583, 17.12817, 37.97046, 33.4..."
ISIC_0015902,0,0.001116,"[65.0, 3.22, 14.2423288822174, 12.164757274256..."
ISIC_0024200,0,0.000005,"[55.0, 2.73, 24.72552, 20.05747, 26.4649, 25.7..."
...,...,...,...
ISIC_9999937,0,0.004151,"[70.0, 6.8, 22.5743345979784, 14.9446663205131..."
ISIC_9999951,0,0.000216,"[60.0, 3.11, 19.97764, 16.02687, 34.15884, 31...."
ISIC_9999960,0,0.000345,"[65.0, 2.05, 17.3325667219097, 12.364397422281..."


In [22]:
# Save to CSV for Analysis
# 'feature_and_probabilities.csv'에는 'target', 'predicted_prob', 'ID마다 Feture Vector'가 포함되어 있다. 
'''
feature_output.to_csv('feature_and_probabilities.csv', index = False)
print("Feature vectors and probabilities have been saved to 'feature_and_probabilities.csv'")
'''

'\nfeature_output.to_csv(\'feature_and_probabilities.csv\', index = False)\nprint("Feature vectors and probabilities have been saved to \'feature_and_probabilities.csv\'")\n'